In [1]:
from datasets import COCODateset
from config import active_config
import random
import numpy as np

In [2]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

class CaptionPreprocessor():
    EOS_TOKEN = 'xeosx'
    
    def __init__(self, clean_descriptions=None):
        self.tokenizer = Tokenizer()
        self.clean_descriptions = clean_descriptions or active_config.clean_descriptions
        
    def fit_captions(self, captions_list):
        if self.clean_descriptions:
            captions_list = self.description_cleaner(captions_list)
        captions_list = self.eos_adder(captions_list)
        self.tokenizer.fit_on_texts(captions_list)
        
    def description_cleaner(self, captions_list):
        # TODO clean descriptions
        return captions_list
        
    def eos_adder(self, captions_list):
        return [caption + ' ' + self.EOS_TOKEN for caption in captions_list]

C:\Users\windo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
class DataProvider():
    def __init__(self, best_caption=False, batch_size=None):
        
        self.best_caption = best_caption
        self.batch_size = batch_size or active_config.batch_size
        
        self.dataset = COCODateset(best_caption)
        self.caption_preprocessor = CaptionPreprocessor()
        
        self.build()
    
    def training_set(self):
        for batch in self.batch_generator(self.dataset.training_set):
            yield batch
        
    def build(self):
        training_all_captions = self.dataset.training_all_captions
        self.caption_preprocessor.fit_captions(training_all_captions)
        
    def batch_generator(self, dataset):
        curr_data_batch = []
        while True:
            idxs = np.random.randint(low=0, high=len(dataset), size=self.batch_size)
            curr_data_batch = dataset[idxs]
            yield self.preprocess_batch(curr_data_batch)
    
    def preprocess_batch(self, data_batch):
        return data_batch

In [4]:
data_provider = DataProvider()

loading annotations into memory...
Done (t=19.67s)
creating index...
index created!
loading annotations into memory...
Done (t=1.23s)
creating index...
index created!


In [17]:
# t = data_provider.caption_preprocessor.tokenizer

In [18]:
batch = next(data_provider.training_set())